In [3]:
import pandas as pd
from ase import Atoms

metadata_path = "train_metadata.csv"

df = pd.read_csv(metadata_path)

In [31]:
from helpers import get_info_from_metadata
import numpy as np
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.analysis.structure_matcher import StructureMatcher
from helpers import calculate_rmsd_pymatgen
from helpers import align_vacuum_to_z_axis
from helpers import find_vacuum_axis_ase
from ase.io import write, read

index = 3270

info = get_info_from_metadata(df, index)

sid = info['meta']['sid']
bulk_src_id = info['meta']['bulk_src_id']
specific_miller = info['meta']['specific_miller']
shift = info['meta']['shift']
top = info['meta']['top']

n_c = info['config']['n_c']
n_vac = info['config']['n_vac']
height = info['config']['height']

tags = info['structures']['true_tags']
true_system_atomic_numbers = info['structures']['true_atomic_nums']
true_system_positions = info['structures']['true_positions']
true_lattice = info['structures']['true_lattice']
ads_pos_relaxed = info['structures']['ads_pos_relaxed']

true_system = Atoms(numbers=true_system_atomic_numbers,
                        positions=true_system_positions,
                        cell=true_lattice,
                        tags=tags,
                        pbc=True)

print("Original cell:", true_system.cell)

vac_axis_idx = find_vacuum_axis_ase(true_system)
print(f"Vacuum axis index: {vac_axis_idx}")

rotated_system = true_system.copy()
v_target = rotated_system.get_cell()[vac_axis_idx]
rotated_system.rotate(v_target, 'z', rotate_cell=True)

if vac_axis_idx == 0:   # x가 진공 -> (y, z, x) 순서로 변경 (Right-handed 유지)
    new_order = [1, 2, 0]
elif vac_axis_idx == 1: # y가 진공 -> (z, x, y) 순서로 변경
    new_order = [2, 0, 1]
else:                   # 이미 z가 진공
    new_order = [0, 1, 2]

# 축이 변경되어야 한다면 순서 교환 수행
if vac_axis_idx != 2:
    old_cell = rotated_system.get_cell()
    new_cell = old_cell[new_order]  # 벡터의 순서만 바꿈 (방향은 그대로)
    
    old_scaled = rotated_system.get_scaled_positions()
    new_scaled = old_scaled[:, new_order] # 좌표값(fractional)의 열 순서 변경
    
    rotated_system.set_cell(new_cell)
    rotated_system.set_scaled_positions(new_scaled)

from numpy.linalg import norm

z_dir = np.array([0, 0, 1])
cell_c = np.array(rotated_system.get_cell()[2])
cos_angle = np.dot(cell_c, z_dir) / (norm(cell_c) * norm(z_dir))
is_parallel = np.isclose(abs(cos_angle), 1.0, atol=1e-6)
print("Is parallel?", is_parallel)

print("Rotated cell:", rotated_system.cell)

write(f"annomaly_samples/{index}_rotated_system.cif", rotated_system)

rmsd_rot = calculate_rmsd_pymatgen(
    struct1=rotated_system,
    struct2=true_system,
    ltol=0.2, stol=0.3, angle_tol=5,
    primitive_cell=False,
)

print(f"RMSD (rotated system): {rmsd_rot}")

# write(f"annomaly_samples//{index}_true_system.cif", true_system)

# true_system = read(f"annomaly_samples/{index}_true_system.cif")

slab_mask = (tags == 0) | (tags == 1)
# true_system = rotated_system
true_slab = true_system[slab_mask]

# Remove vacuum in supercell slab
n_layers = n_c + n_vac

print("num layers of vacuum:", n_vac)
print("num layers of slab:", n_c)
print("height:", height)

write(f"{index}_true_system.cif", true_system)

Original cell: Cell([[13.77790641784668, 0.0, 1.7953521013259888], [5.724344253540039, 30.910184860229492, 2.1544225215911865], [0.0, 0.0, 38.40353012084961]])
Vacuum axis index: 1
Is parallel? True
Rotated cell: Cell([[-6.9767835570841195, -37.67307833485302, 2.6257923978547746], [13.026118049994004, -4.059489855405834, 2.6257922781808047], [4.735031473208967e-16, -8.768938240440446e-17, 31.50950938412302]])
RMSD (rotated system): (np.float64(8.795965848667439e-16), np.float64(1.7498803117603598e-15))
num layers of vacuum: 4
num layers of slab: 2
height: 5.151697360147734


In [15]:
from pymatgen.core import Structure, Lattice
from pymatgen.io.ase import AseAtomsAdaptor
import math 

adaptor = AseAtomsAdaptor()

scaling_factor = n_layers / n_c

tight_slab = true_slab.copy()
tight_cell = tight_slab.get_cell()
tight_cell[2] = tight_cell[2] * (n_c / n_layers)
tight_slab.set_cell(tight_cell)

tight_slab.center() # This is very important!

# Get primitive slab

tight_slab_struct = adaptor.get_structure(tight_slab)

prim_slab_struct = tight_slab_struct.get_primitive_structure(tolerance=0.1)
sc_matrix = np.round(np.dot(
    tight_slab_struct.lattice.matrix,
    np.linalg.inv(prim_slab_struct.lattice.matrix)
)).astype(int)
print(f"Primitive atoms: {len(prim_slab_struct)} (original: {len(tight_slab_struct)})")
print(f"Supercell matrix:\n{sc_matrix}")

# Reconstruct tight slab from primitive slab

recon_struct = prim_slab_struct.copy()
recon_struct.make_supercell(sc_matrix, to_unit_cell=False)

recon_tight_slab = adaptor.get_atoms(recon_struct)

rmsd_tight = calculate_rmsd_pymatgen(
    struct1=recon_tight_slab,
    struct2=tight_slab,
    ltol=0.2, stol=0.3, angle_tol=5,
    primitive_cell=False,
)

# Reconstruct slab with vacuum

recon_slab = recon_tight_slab.copy()
recon_cell = recon_slab.get_cell()
recon_cell[2] = recon_cell[2] * scaling_factor
recon_slab.set_cell(recon_cell)

# recon_slab.center()

rmsd_w_vac = calculate_rmsd_pymatgen(
    struct1=recon_slab,
    struct2=true_slab,
    ltol=0.2, stol=0.3, angle_tol=5,
    primitive_cell=False,
)
print(f"\n RMSD (with vacuum): {rmsd_w_vac}")


Primitive atoms: 3 (original: 54)
Supercell matrix:
[[ 0  1 -3]
 [-2 -1  0]
 [-2  2  0]]

 RMSD (with vacuum): (np.float64(1.1495800048771005e-07), np.float64(2.418619793558546e-07))


In [28]:
# Add adsorbate to recon_slab

from ase import Atoms

adsorbate = true_system[tags == 2]
diff_vector = recon_slab.get_center_of_mass() - true_slab.get_center_of_mass()
adsorbate.translate(diff_vector)

rmsd_system, _ = calculate_rmsd_pymatgen(
        struct1=recon_slab + adsorbate,
        struct2=true_system,
        ltol=0.2, stol=0.3, angle_tol=5,
        primitive_cell=False,
)

# adsorbate.positions = ads_pos_relaxed

# diff_vector = recon_slab.get_center_of_mass() - true_slab.get_center_of_mass()
# adsorbate.translate(diff_vector)


recon_system = recon_slab + adsorbate

# rmsd_system = calculate_rmsd_pymatgen(
#         struct1=recon_system,
#         struct2=true_system,
#         ltol=0.2, stol=0.3, angle_tol=5,
#         primitive_cell=False,
#         )
print("RMSD (system) : ", rmsd_system)

RMSD (system) :  None


In [8]:
# What should the model learn?
# ==> primitive slab's structure (including atom types, coordinates, lattice vectors)
# ==> supercell matrix
# ==> scaling factor (I think this cannot be not included in the supercell matrix.)
# ==> adsorbate coordinates

In [33]:
from ase.visualize import view
from ase.io import write

view(recon_system, viewer='x3d')
# view(true_system, viewer='ngl')

# write(f"{index}_recon_system.cif", recon_system)
# write(f"{index}_true_system.cif", true_system)